## Lopputyö: Pauli twirling

Paulin pyöräytys (Pauli twirling), joka tunnetaan myös satunnaistettuna kokoamisena (randomized compiling), on yleinen askel monien melunvaimennustekniikoiden edellä, joita käytetään kvanttilaskennassa. Paulin pyöräytyksellä on mahdollista muokata yleiset melukanavat stokastisiksi Paulin kanaviksi, joiden malli on kohtalaisen yksinkertainen, samalla kun paikalliset piirit pysyvät muuttumattomina. 

Tässä viimeisessä projektissa tutkimme Paulin pyöräytyksen vaikutusta erilaisiin melukanaviin, joita tällä kurssilla on käsitelty, toteutamme mukautetun koeluokan, jota voidaan käyttää yhdessä muiden qiskit-koeluokkien kanssa ja simuloimme Paulin pyöräytyksen vaikutusta melukanaviin.

Paulin pyöräytys esitettiin ensimmäistä kertaa satunnaistettuna kokoamisena tutkimuksessa [J. Wallman and J. Emerson, Phys. Rev. A 94, 052325 (2015)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.052325). Se toimii tuottamalla klassisia sekoituksia melukanavista käyttäen portteja kanavan jommassakummassa päädyssä. Nämä portit valitaan usein olemaan Paulin portteja $\{I, X, Y, Z\}$. Pyöräytyksen toimintaa (twirling action) $\mathcal{T}_i$ kvanttikanavaan $\Lambda(\rho)$ kuvaa yhtälö: 
\begin{align}
\mathcal{T}_i\Lambda &=\mathcal{P}_i\Lambda\mathcal{P}_i^\dagger
\end{align}
jossa superoperaattori $\mathcal{P}_i$ toimii operaattoriin seuraavalla tavalla: 
\begin{align}
\mathcal{P}_i\Lambda&= \sigma_i^\dagger\Lambda\sigma_i.
\end{align}

Lopullinen kanava saadaan laskemalla keskiarvo pyöräytettyjen kanavien ja valittujen porttien yli, tässä tapauksessa Paulin porttien yli: 
\begin{align}
\widetilde{\Lambda} &= \frac{1}{4}\sum_{i=0}^3{\mathcal{T}_i\Lambda} \\
&= \frac{\Lambda + \mathcal{P}_{x}\Lambda \mathcal{P}_{x}^\dagger + \mathcal{P}_{y}\Lambda \mathcal{P}_{y}^\dagger + \mathcal{P}_{z} \Lambda \mathcal{P}_{z}^\dagger}{4}.
\end{align}

\begin{align}
\widetilde{\Lambda}(\rho) &= \frac{1}{4}\sum_{i=0}^3{\mathcal{T}_i\Lambda(\rho)} \\
&= \frac{1}{4}\sum_{i=0}^3{\mathcal{P}_i(\Lambda(\mathcal{P}_i^\dagger(\rho)))} \\
&= \frac{\Lambda \rho + \mathcal{P}_{x}\Lambda \mathcal{P}_{x}^\dagger \rho + \mathcal{P}_{y}\Lambda \mathcal{P}_{y}^\dagger \rho + \mathcal{P}_{z} \Lambda \mathcal{P}_{z}^\dagger \rho}{4}.
\end{align}

Aloitamme tiheysmatriisista $\rho$ ja sovellamme kanavaa $\Lambda$  siihen: :
$$\Lambda \rho = \Lambda(\rho)$$
Seuraavaksi pyöräytämme ainoastaan kanavaa: 
$$\widetilde{\Lambda} = \mathcal{T}_i\Lambda = \mathcal{P}_i\Lambda\mathcal{P}_i^\dagger$$
Kun sovellamme tätä uutta kanavaa $\rho$:n uudelleen, saamme 
$$\widetilde{\Lambda}(\rho) = \mathcal{P}_i\Lambda\mathcal{P}_i^\dagger\rho = \mathcal{P}_i(\Lambda(\mathcal{P}_i^\dagger(\rho))).$$
Oikeanpuoleinen Paulin superoperaattori toimii pyöräytetyn kanavan sisällä näkyvään kohteeseen, ei itse kanavaan. 

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import numpy as np

### Tehtävä 1 (7p)

Laske algebrallisesti lopputuloksena saatava kanava, kun seuraaviin kohteisiin käytetään Paulin pyöräytystä: 

1) Bitinkääntökanava (bit-flip channel) 
$$ \Lambda_d (\rho) = (1-p) I \rho I + p (X \rho X)$$
2) Koherentti kierto (coherent rotation)
$$ \Lambda_r (\rho) = \cos^2{\left(\frac{\theta}{2}\right)} I \rho I  + \sin^2{\left(\frac{\theta}{2}\right)} X \rho X + \frac{i}{2} (\sin{(\theta)} I \rho X - \sin{(\theta)} X \rho I)$$
3) Amplitudinvaimennuskanava (amplitude damping channel)
$$ \Lambda_a (\rho) = A_0 \rho A_0^\dagger + A_1 \rho A_1^\dagger$$
jossa $A_0 = \frac{(1+\sqrt{1-p})}{2} I  + \frac{(1-\sqrt{1-p})}{2} Z$ ja $A_1 = \frac{\sqrt{p}}{2} (X + iY)$.

Kirjoita lopputuloksena saatavat kanavat Paulin kanavina.

### Tehtävä 2 (8p)

Kirjoita mukautettuja koe- ja analyysiluokkia, joilla saat poimittua tiheysmatriisit tilatomografian avulla melukanavien pyöräyttämisen jälkeen.  
Voit käyttää tähän esimerkiksi qiskit-experiments-kehystä. Koeluokan tulisi hyväksyä syötteeksi vähintään yksi melukanavalla varustettu piiri, ja analyysiluokan tulisi palauttaa lopputuloksena tiheysmatriisi. 

Tämän voi toteuttaa monella eri tavalla, mutta alla on malli, josta voi olla apua, jos miettii mistä aloittaa. Koeluokkasi pohjana voi olla `BaseExperiment` tai `TomographyExperiment` ja se voi käyttää `__init__` ja  `circuit` -metodeja tuottaakseen Paulin pyöräytykseen tarvittavat piirit. Analyysiluokan pohjana taas voi olla `BaseAnalysis` ja se voi käyttää `_run_analysis` -metodia ottaakseen tuloksista keskiarvon erilaisten pyöräytysten yli, joita kokeessa saadaan. Voit käyttää seuraavaa ohjetta apunasi: [qiskit-experiments Documentation](https://qiskit.org/ecosystem/experiments/tutorials/custom_experiment.html). Qiskit-experiments käyttää täysin avointa lähdekoodia, joten lähdekoodin tarkastelu voi myös olla hyödyksi. 

In [46]:
from qiskit import QuantumCircuit
from typing import Tuple, List, Optional, Sequence
import matplotlib
from qiskit.providers.backend import Backend
from qiskit_experiments.library.tomography import TomographyExperiment
from qiskit_experiments.framework import (
    BaseAnalysis,
    Options,
    ExperimentData,
    AnalysisResultData
)

class TwirledStateTomographyAnalysis(BaseAnalysis):
    """Analysis class for state tomography with twirling."""

    @classmethod
    def _default_options(cls) -> Options:
        """Set default analysis options."""

        options = super()._default_options()
        options.plot = False
        options.ax = None
        return options

    def _run_analysis(
        self,
        experiment_data: ExperimentData
    ) -> Tuple[List[AnalysisResultData], List["matplotlib.figure.Figure"]]:
        """Run the analysis."""

        data_objects = experiment_data.data()

        # Combine twirled results and perform state tomography
        result = AnalysisResultData()

        return result, []

class TwirledStateTomography(TomographyExperiment):
    """State tomography with twirling."""

    def __init__(self,
                 circuit: QuantumCircuit,
                #  add your own arguments here...
                 physical_qubits: Sequence[int] = None,
                 measurement_indices: Sequence[int] = None,
                 backend: Optional[Backend] = None):
        """Initialize the experiment."""
        if physical_qubits is None:
            physical_qubits = tuple(range(circuit.num_qubits))
        super().__init__(circuit=circuit,
                        backend=backend,
                        physical_qubits=physical_qubits,
                        measurement_indices=measurement_indices,
                        analysis=TwirledStateTomographyAnalysis(),
                        )

    def circuits(self) -> List[QuantumCircuit]:
        """Generate the list of circuits to be run."""
        circuits = []
        # Generate circuits and populate metadata here

        return circuits

    @classmethod
    def _default_experiment_options(cls) -> Options:
        """Set default experiment options here."""
        options = super()._default_experiment_options()
        return options

### Tehtävä 3 (8p)
Alusta kubitti tilaan, jossa populaatiot ja koherenssit poikkeavat nollasta. Käytä jokaista alla listattua melukanavaa tähän kubittiin ja tee tilatomografia sekä pyöräyttäen kanavaa että ilman pyöräytystä. Piirrä kullekin melukanavalle samaan kuvaajaan $\rho_{00}$, $\rho_{11}$, $Re(\rho_{01})$ ja $Im(\rho_{01})$ sekä pyöräytetylle että pyöräyttämättömälle tapaukselle, eri $p$:n ja $\theta$:n arvoille.   

Melukanavat :
- Bitinkääntö 
    - Paulin kanava arvoilla $(1-p, p, 0, 0)$.
- Koherentti kierto 
    - Käytä RX-porttia pienellä kulmalla.
- Amplitudinvaimennuskanava 


Voit käyttää näiden kanavien toteutuksia aiemmista projekteista tai hyödyntää `qiskit_aer.noise`:n melumalleja. 

In [ ]:
p_values = np.linspace(0, 1, 10)
theta_values = np.linspace(-np.pi, np.pi, 10)



### Tehtävä 4 (7p)
Viimeisenä, käännetään tai vaimennetaan pyöräytetty melukanava, jotta saadaan tiheysmatriisi ilman melua. Pyöräyttämisen jälkeen melukanavan tulisi olla Paulin kanavan muodossa, joka on suhteellisen helppo kääntää, jos melukertoimet tunnetaan. Käännetty kanava ei välttämättä ole fysikaalinen, sillä kertoimet voivat olla negatiivisia. Menetelmä tavallisten melukanavien kääntämiseksi kuvaillaan tutkimuksessa [S. Mangini, et al. EPJ Quantum Technol. 9, 29 (2022)](https://doi.org/10.1140/epjqt/s40507-022-00151-0). 

Yleisen Paulin kanavan käänteiskanava kertoimilla $(p_{0}, p_{x}, p_{y}, p_{z})$ on 
$$\begin{aligned} & \Lambda^{-1}_{\boldsymbol{p}}(\Lambda) = \beta _{0} \Lambda + \beta _{1} \sigma _{x} {\Lambda} \sigma _{x} + \beta _{2} \sigma _{y} {\Lambda} \sigma _{y} + \beta _{3} \sigma _{z} {\Lambda} \sigma _{z}\quad \text{with} \\ & \beta _{0} =\frac{1}{4} \biggl(1+\frac{1}{1-2(p_{x}+p_{y})}+ \frac{1}{1-2(p_{x}+p_{z})}+\frac{1}{1-2(p_{y}+p_{z})} \biggr), \\ & \beta _{1} = \frac{1}{4} \biggl(1-\frac{1}{1-2(p_{x}+p_{y})}- \frac{1}{1-2(p_{x}+p_{z})}+\frac{1}{1-2(p_{y}+p_{z})} \biggr), \\ & \beta _{2} = \frac{1}{4} \biggl(1-\frac{1}{1-2(p_{x}+p_{y})}+ \frac{1}{1-2(p_{x}+p_{z})}-\frac{1}{1-2(p_{y}+p_{z})} \biggr), \\ & \beta _{3} = \frac{1}{4} \biggl(1+\frac{1}{1-2(p_{x}+p_{y})}- \frac{1}{1-2(p_{x}+p_{z})}-\frac{1}{1-2(p_{y}+p_{z})} \biggr) . \end{aligned}$$

Käyttäen tehtävän 1 pyöräytettyjen melukanavien kertoimia, käännä kaikki tehtävässä 3 löytämäsi tiheysmatriisit (jokaiselle $p$ ja $\theta$ arvolle) ja vertaa termejä $\rho_{00}$, $\rho_{11}$, $Re(\rho_{01})$ ja $Im(\rho_{01})$ kääntämättömiin versioihin esimerkiksi kuvaajien avulla. 

Vihje: Kaavat yksinkertaistuvat huomattavasti, jos sijoitat niihin vakiot tehtävästä 1. 